# Algorithm Cage Match

This notebook generates sets of fake STTCs from PTCs, and determines the mean square error produced by TEPs-I, Bagheri and my count matching algorithms.

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd
from traffic_prophet import cfg
import pathlib, os
import configparser

from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import growthfactor as gf
from traffic_prophet.countmatch import neighbour

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

filepath = pathlib.Path.home().joinpath('.charlesconfig')
if os.path.isfile(filepath):
    vol_conn = connection.Connection(filepath, 'POSTGRES',
                                     'czhu.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'POSTGRES',
                                    'czhu.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['MAPBOX']['token']
    PLOTLY_USER = config['PLOTLY']['user']
    PLOTLY_KEY = config['PLOTLY']['key']
else:
    filepath = pathlib.Path.home().joinpath('cf.txt')
    vol_conn = connection.Connection(filepath, 'localpg',
                                     'prj_vol.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'localpg',
                                    'gis.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['mapbox']['token']
    PLOTLY_USER = config['plotly']['user']
    PLOTLY_KEY = config['plotly']['apikey']

In [2]:
rdr = reader.Reader(vol_conn)
%time rdr.read()

CPU times: user 1min 35s, sys: 2.09 s, total: 1min 37s
Wall time: 1min 47s


In [3]:
gf.get_growth_factors(rdr)

In [4]:
ptc_ids = np.unique(np.abs(list(rdr.ptcs.keys())))
nb = neighbour.NeighbourLonLatEuclidean(ll_conn, 20, ptc_ids)
%time nb.find_neighbours()

CPU times: user 18.4 s, sys: 578 ms, total: 19 s
Wall time: 18.7 s


## STTC-MC generator

This routine randomly generates a set of STTCs 

In [8]:
p = rdr.ptcs[-30022069]

In [11]:
doyr = p.data['Daily Count'].loc[:, ['Date']].copy()

In [12]:
    year = 2010    
    doyr.loc[year, 'Day-to-AADT Ratio'] = (
            p.data['AADT'].at[year, 'AADT'] /
            p.data['Daily Count'].loc[year, 'Daily Count']).values

In [13]:
doyr

Date  Day-to-AADT Ratio
Year Day of Year                              
2010 1           2010-01-01           1.439357
     2           2010-01-02           1.098346
     3           2010-01-03           1.284496
     6           2010-01-06           0.952754
     7           2010-01-07           0.939115
     8           2010-01-08           0.896251
     9           2010-01-09           0.938695
     10          2010-01-10           1.058197
     11          2010-01-11           0.926145
     12          2010-01-12           0.896869
     16          2010-01-16           0.895644
     17          2010-01-17           1.046237
     18          2010-01-18           0.946408
     19          2010-01-19           0.896517
     20          2010-01-20           0.899928
     21          2010-01-21           0.860825
     22          2010-01-22           0.818632
     23          2010-01-23           0.877987
     24          2010-01-24           1.060220
     25          2010-01-25           0.937889
     26          2010-01-26           0.875744
     27          2010-01-27           0.879819
     28          2010-01-28           0.854778
     29          2010-01-29           0.825258
     30          2010-01-30           0.932706
     32          2010-02-01           0.915146
     33          2010-02-02           0.896400
     34          2010-02-03           0.897690
     35          2010-02-04           0.853772
     38          2010-02-07           1.061159
...                     ...                ...
2017 326         2017-11-22                NaN
     329         2017-11-25                NaN
     330         2017-11-26                NaN
     331         2017-11-27                NaN
     332         2017-11-28                NaN
     333         2017-11-29                NaN
     334         2017-11-30                NaN
     335         2017-12-01                NaN
     336         2017-12-02                NaN
     337         2017-12-03                NaN
     338         2017-12-04                NaN
     339         2017-12-05                NaN
     340         2017-12-06                NaN
     341         2017-12-07                NaN
     342         2017-12-08                NaN
     343         2017-12-09                NaN
     345         2017-12-11                NaN
     346         2017-12-12                NaN
     347         2017-12-13                NaN
     349         2017-12-15                NaN
     350         2017-12-16                NaN
     351         2017-12-17                NaN
     352         2017-12-18                NaN
     353         2017-12-19                NaN
     355         2017-12-21                NaN
     356         2017-12-22                NaN
     357         2017-12-23                NaN
     358         2017-12-24                NaN
     359         2017-12-25                NaN
     363         2017-12-29                NaN

[878 rows x 2 columns]

In [26]:
import importlib

In [31]:
import countmatch_teps as cmt
importlib.reload(cmt)

<module 'countmatch_teps' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_teps.py'>

In [33]:
cmt.estimate_aadts(rdr, progress_bar=False)

In [16]:
cmt.mse_preprocess_ptc_loop(rdr.ptcs[-30022069])

In [17]:
rdr.ptcs[-30022069].data['Day-to-AADT Factors']

,Year,Day of Year,Date,Day-to-AADT Ratio,Month,Day of Week
0,2010,1,2010-01-01,1.439357,1,4
1,2010,2,2010-01-02,1.098346,1,5
2,2010,3,2010-01-03,1.284496,1,6
3,2010,6,2010-01-06,0.952754,1,2
4,2010,7,2010-01-07,0.939115,1,3
5,2010,8,2010-01-08,0.896251,1,4
6,2010,9,2010-01-09,0.938695,1,5
7,2010,10,2010-01-10,1.058197,1,6
8,2010,11,2010-01-11,0.926145,1,0
9,2010,12,2010-01-12,0.896869,1,1


In [19]:
rdr.ptcs[-30022069].data['MSE Annual-DoW Averages']

Day-to-AADT Ratio     MADT Avg.  DoM Factor Avg.  \
Day of Week Year                                                     
0           2010           1.042476  75015.629732         1.022638   
            2016           1.064127  66850.819120         1.033885   
            2017           1.031848  75671.177854         1.025442   
1           2010           0.992354  75292.119551         0.978459   
            2016           1.035474  66614.166554         0.999164   
            2017           1.003588  75447.645380         0.999258   
2           2010           0.976181  75543.003188         0.965887   
            2016           1.028342  66561.915694         0.993266   
            2017           0.977590  75627.310892         0.976077   
3           2010           0.973056  75420.111113         0.958091   
            2016           1.014107  67511.293596         0.993883   
            2017           0.978881  75729.129550         0.977775   
4           2010           0.982548  74845.761891         0.951136   
            2016           1.012176  66376.474871         0.974073   
            2017           0.956091  75530.025640         0.951447   
5           2010           1.021226  75916.168541         1.014062   
            2016           0.998375  67657.838260         0.993955   
            2017           1.011302  75558.422374         1.004674   
6           2010           1.190005  75134.017748         1.160853   
            2016           1.028835  68378.753404         1.036952   
            2017           1.092886  75609.479395         1.082752   

                   DoMADT Avg.  
Day of Week Year                
0           2010  73414.650000  
            2016  64779.613636  
            2017  73914.500000  
1           2010  76986.789474  
            2016  67060.361702  
            2017  75551.300000  
2           2010  78203.948718  
            2016  67398.125000  
            2017  77527.405405  
3           2010  78802.574468  
            2016  68285.347826  
            2017  77526.611111  
4           2010  78756.547619  
            2016  68459.130435  
            2017  79450.523810  
5           2010  74974.348837  
            2016  67826.362791  
            2017  75300.355556  
6           2010  64953.342105  
            2016  65593.333333  
            2017  69973.150000

In [20]:
johnny = False

In [23]:
not johnny

True

In [41]:
np.mean([v.growth_factor for v in rdr.ptcs.values() if v.data['AADT'].shape[0] > 1])

0.9938382604854958

In [42]:
import countmatch_common as cmc

In [43]:
importlib.reload(cmc)

<module 'countmatch_common' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_common.py'>

In [45]:
cmc.get_citywide_growth_factor(rdr, multi_year=True)

0.9938382604854958